In [62]:
import os
import sys
import numpy as np
import cv2

from pathlib import Path
import shutil
import pickle

In [2]:
root_path = '/home/aiteam/tykim/generative_model/human/EVA3D/datasets'
sub_list = ['maria', 'zhuna', 'soufianou_boubacar_moumouni', 'natacha', 'mahaoran', 'joseph_matanda', 'jinyutong', 'fuzhizhi', 'barry', 'amanda']


In [6]:
dest_gene_path = Path(root_path).joinpath('Genebody')

In [7]:
dest_gene_path.mkdir(parents=True, exist_ok=True)

In [39]:
dest_img_path = dest_gene_path.joinpath('images')
dest_segm_path = dest_gene_path.joinpath('segm')

dest_img_path.mkdir(parents=True, exist_ok=True)
dest_segm_path.mkdir(parents=True, exist_ok=True)

In [9]:
each_sub_path = Path(root_path).joinpath(sub_list[0])
each_sub_path

PosixPath('/home/aiteam/tykim/generative_model/human/EVA3D/datasets/maria')

In [19]:
# 전체 데이터중에서 n개만 얻기 : 인덱스 얻기
total_list = sorted(list(each_sub_path.joinpath('param').rglob('*.npy')))

In [27]:
start = int(total_list[0].name.replace('.npy', ''))
end = int(total_list[-1].name.replace('.npy', ''))

In [30]:
idx_list = np.linspace(start, stop=end, num=10, dtype=np.int32)

In [31]:
camera_list = ['01', '13', '25', '37']

In [32]:
cam_id = camera_list[0]

In [40]:
test = {}

In [42]:
test['a'] = {}

In [45]:
test['a']['b'] = 100
test['a']['c'] = 200

In [46]:
test

{'asdasd': {}, 'a': {'b': 100, 'c': 200}}

In [63]:
smpl = {}


for idx in idx_list:
    img_path = each_sub_path.joinpath('image', cam_id, f'0{str(idx)}.jpg')
    seg_path = each_sub_path.joinpath('mask', cam_id, f'mask0{str(idx)}.png')
    smpl_param_path = each_sub_path.joinpath('param',  f'0{str(idx)}.npy')

    dest_img = dest_img_path.joinpath(f'{str(idx)}_{cam_id}_{sub_list[0]}.jpg')
    dest_seg = dest_segm_path.joinpath(f'mask0{str(idx)}_{cam_id}_{sub_list[0]}.png')


    # get smpl from gene
    smpl_data = np.load(smpl_param_path, allow_pickle=True).item()
    smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}'] = {}
    smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['body_pose'] = smpl_data['smplx']['body_pose'].reshape(1, -1) #reshape
    smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['global_orient'] = smpl_data['smplx']['global_orient']
    smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['betas'] = smpl_data['smplx']['betas']

    # ['camera_rotation'], ['camera_translation']

    # get cam anno from gene
    annotation = np.load(each_sub_path.joinpath('annots.npy'), allow_pickle=True)

    c2w_R = annotation.item()['cams'][cam_id]['c2w_R']
    c2w_T = annotation.item()['cams'][cam_id]['c2w_T']

    R_smplx = np.matmul(np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]]), c2w_R, dtype=np.float32)
    R_smplx = np.matmul(np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]]), R_smplx,  dtype=np.float32)
    
    smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['camera_rotation'] = R_smplx
    smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['camera_translation'] = c2w_T.astype(np.float32)

    shutil.copy2(img_path, dest_img)
    shutil.copy2(seg_path, dest_seg)

with open(dest_gene_path.joinpath('smpl.pkl'), 'wb') as f:
    pickle.dump(smpl, f)

In [ ]:
with open('/home/aiteam/tykim/generative_model/human/EVA3D/datasets/DeepFashion/smpl.pkl', 'rb') as f:
    data = pickle.load(f)

In [69]:
sub_list = ['maria']

In [70]:
# 8 * 10 * 10 = 800 장
smpl = {}
camera_list = ['01', '13', '25', '37', '07', '19', '31', '43']

dest_gene_path = Path(root_path).joinpath('Genebody')
dest_img_path = dest_gene_path.joinpath('images')
dest_segm_path = dest_gene_path.joinpath('segm')

dest_img_path.mkdir(parents=True, exist_ok=True)
dest_segm_path.mkdir(parents=True, exist_ok=True)


for sub_name in sub_list:
    each_sub_path = Path(root_path).joinpath(sub_name)
    # 전체 데이터중에서 n개만 얻기 : 인덱스 얻기
    total_list = sorted(list(each_sub_path.joinpath('param').rglob('*.npy')))

    start = int(total_list[0].name.replace('.npy', ''))
    end = int(total_list[-1].name.replace('.npy', ''))

    idx_list = np.linspace(start, stop=end, num=10, dtype=np.int32)

    for cam_id in camera_list:
        for idx in idx_list:
            img_path = each_sub_path.joinpath('image', cam_id, f'0{str(idx)}.jpg')
            seg_path = each_sub_path.joinpath('mask', cam_id, f'mask0{str(idx)}.png')
            smpl_param_path = each_sub_path.joinpath('param',  f'0{str(idx)}.npy')

            dest_img = dest_img_path.joinpath(f'{str(idx)}_{cam_id}_{sub_list[0]}.jpg')
            dest_seg = dest_segm_path.joinpath(f'mask0{str(idx)}_{cam_id}_{sub_list[0]}.png')


            # get smpl from gene
            smpl_data = np.load(smpl_param_path, allow_pickle=True).item()
            smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}'] = {}
            smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['body_pose'] = smpl_data['smplx']['body_pose'].reshape(1, -1) #reshape
            smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['global_orient'] = smpl_data['smplx']['global_orient']
            smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['betas'] = smpl_data['smplx']['betas']

            # ['camera_rotation'], ['camera_translation']

            # get cam anno from gene
            annotation = np.load(each_sub_path.joinpath('annots.npy'), allow_pickle=True)

            c2w_R = annotation.item()['cams'][cam_id]['c2w_R']
            c2w_T = annotation.item()['cams'][cam_id]['c2w_T']

            R_smplx = np.matmul(np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]]), c2w_R, dtype=np.float32)
            R_smplx = np.matmul(np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]]), R_smplx,  dtype=np.float32)
            
            smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['camera_rotation'] = R_smplx
            smpl[f'{str(idx)}_{cam_id}_{sub_list[0]}']['camera_translation'] = c2w_T.astype(np.float32)

            shutil.copy2(img_path, dest_img)
            shutil.copy2(seg_path, dest_seg)

In [71]:
with open(dest_gene_path.joinpath('smpl.pkl'), 'wb') as f:
    pickle.dump(smpl, f)

In [74]:
with open('/home/aiteam/tykim/generative_model/human/EVA3D/datasets/Genebody/smpl.pkl', 'rb') as f:
    data = pickle.load(f)

In [76]:
len(list(data.keys()))

80